In [19]:
%reset -f

import random, operator
import numpy as np
from numpy import sqrt
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats
import plotly
import plotly.graph_objs as go
from plotly import subplots

In [87]:
# PARAMETERS
sims = 1
rounds = 1000


I = 50                           #type 1 population
J = I                              #type 2 population

mutation_rate = 0.01


max_production = 2
lambdaa = 0.5
rho = 1.2


In [24]:
def randominitialize_type1(popl_size):
    pop = []
    for i in range(popl_size):
        pair = []
        pair.append(random.uniform(0,max_production))
        pair.append(random.randint(0,1))
        
        pop.append(pair)
    return pop
        
    

In [25]:
def randominitialize_type2(popl_size):
    pop = []
    for i in range(popl_size):
        pop.append(random.uniform(0,max_production))       
    return pop

In [26]:
def matching_and_utility(normalized_pop1, normalized_pop2):
#     random.shuffle(normalized_pop1)
#     random.shuffle(normalized_pop2)
        
    type1_utilities = []
    type2_utilities = []
    for n in range(len(normalized_pop1)):
        
        type1_utilities.append(2*sqrt(min(normalized_pop2[n],normalized_pop1[n][0])) - min(normalized_pop2[n],normalized_pop1[n][0]) + (1-normalized_pop1[n][1])*lambdaa*rho*min(normalized_pop2[n],normalized_pop1[n][0]))
        type2_utilities.append(-min(normalized_pop2[n],normalized_pop1[n][0]) + normalized_pop1[n][1]*rho*2*sqrt(min(normalized_pop2[n],normalized_pop1[n][0])))
        
#         type1_utilities.append(2*(min(normalized_pop1[n][0],normalized_pop2[n]))**(1/2) - min(normalized_pop1[n][0],normalized_pop2[n]) + (1-normalized_pop1[n][1])*lambdaa*rho*min(normalized_pop1[n][0],normalized_pop2[n]))
#         type2_utilities.append(-(min(normalized_pop1[n][0],normalized_pop2[n])) + rho*normalized_pop1[n][1]*2*((min(normalized_pop1[n][0],normalized_pop2[n]))**(1/2)))
    
    return type1_utilities, type2_utilities
        
        
    

In [29]:
def reproduction(normalized_pop, fitness_pop):
    pop = []
    for n in range(len(normalized_pop)):
        r = random.sample([k for k in range(len(normalized_pop))], 2)
        r1 = r[0]
        r2 = r[1]
        if fitness_pop[r1] >= fitness_pop[r2]:
            pop.append(normalized_pop[r1])
        else:
            pop.append(normalized_pop[r2])
    return pop
        

In [34]:
def mutation_type1(normalized_pop, mutation_rate):
    pop = []
    for n in range(len(normalized_pop)):
        r = random.uniform(0,1)
        if r <= mutation_rate:
            normalized_pop[n][0] = random.uniform(0,max_production)
            normalized_pop[n][1] = 1 - normalized_pop[n][1]
        else:
            pass
        pop.append(normalized_pop[n])
    return pop

In [35]:
def mutation_type2(normalized_pop, mutation_rate):
    pop = []
    for n in range(len(normalized_pop)):
        r = random.uniform(0,1)
        if r <= mutation_rate:
            normalized_pop[n] = random.uniform(0,max_production)
        else:
            pass
        pop.append(normalized_pop[n])
    return pop

In [90]:
all_runs_type1_average_production = [[0 for s in range(sims)] for t in range(rounds)]
all_runs_type1_average_reneging = [[0 for s in range(sims)] for t in range(rounds)]
all_runs_type2_average_production = [[0 for s in range(sims)] for t in range(rounds)]

all_runs_type1_average_utility = [[0 for s in range(sims)] for t in range(rounds)]
all_runs_type2_average_utility = [[0 for s in range(sims)] for t in range(rounds)]

## note: change these to matrices for better data storage and usage


for s in range(sims):
    pop_type1 = randominitialize_type1(I)
    pop_type2 = randominitialize_type2(J)
    for t in range(rounds):            
        
        utility_type1, utility_type2  = matching_and_utility(pop_type1, pop_type2)
        
        ### storing data ###
        this_round_type1_average_production = np.mean([pop_type1[i][0] for i in range(len(pop_type1))])
        this_round_type1_average_reneging = np.mean([pop_type1[i][1] for i in range(len(pop_type1))])
        this_round_type2_average_production = np.mean([pop_type2[i] for i in range(len(pop_type1))])
        
        this_round_type1_average_utility = np.mean([utility_type1[i] for i in range(len(pop_type1))])
        this_round_type2_average_utility = np.mean([utility_type2[i] for i in range(len(pop_type1))])
                
        all_runs_type1_average_production[t][s] = this_round_type1_average_production
        all_runs_type1_average_reneging[t][s]   = this_round_type1_average_reneging
        all_runs_type2_average_production[t][s] = this_round_type2_average_production
        
        all_runs_type1_average_utility[t][s] = this_round_type1_average_utility
        all_runs_type2_average_utility[t][s] = this_round_type2_average_utility
        ### END - storing data ###
        
        
        
        
        temp_pop_type1 = reproduction(pop_type1, utility_type1)
        temp_pop_type2 = reproduction(pop_type2, utility_type2)
        
        temp_pop_type1 = mutation_type1(temp_pop_type1, mutation_rate)
        temp_pop_type2 = mutation_type2(temp_pop_type2, mutation_rate)
        
        
        pop_type1 = temp_pop_type1 #temp_pop_type1.copy()
        pop_type2 = temp_pop_type2 #temp_pop_type2.copy()
        
average_all_runs_type1_average_production = [0 for t in range(rounds)]
average_all_runs_type1_average_reneging = [0 for t in range(rounds)]
average_all_runs_type2_average_production = [0 for t in range(rounds)]
average_all_runs_type1_average_utility = [0 for t in range(rounds)]
average_all_runs_type2_average_utility = [0 for t in range(rounds)]
        
for t in range(rounds):
    average_all_runs_type1_average_production[t] = np.mean(all_runs_type1_average_production[t])
    average_all_runs_type1_average_reneging[t]   = np.mean(all_runs_type1_average_reneging[t])
    average_all_runs_type2_average_production[t] = np.mean(all_runs_type2_average_production[t])
    average_all_runs_type1_average_utility[t]    = np.mean(all_runs_type1_average_utility[t])
    average_all_runs_type2_average_utility[t]    = np.mean(all_runs_type2_average_utility[t])
    

    
### GRAPHING ###
trace1 = go.Scatter(x=list(range(rounds)), y=list(average_all_runs_type1_average_production),
                    mode='lines', name='Average type 1 production')               
layout1 = go.Layout(
    title=('Production'),
    width=800,
    height=600,
    xaxis=dict(
        title='round',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Production',
        nticks=10,
        range=[0,max_production+0.25],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
data1 = [trace1]
figure1 = go.Figure(data=data1,layout=layout1)
figure1.add_trace(go.Scatter(x=list(range(rounds)), y=list(average_all_runs_type2_average_production),
                    mode='lines', name='Average type 2 production'))
figure1.add_trace(go.Scatter(x = list(range(rounds)),y = list(average_all_runs_type1_average_reneging), mode = 'lines',name = 'Average type 1 reneging portion'))

figure1.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
figure1.show()  






### GRAPH 2
trace2 = go.Scatter(x=list(range(rounds)), y=list(average_all_runs_type1_average_utility),
                    mode='lines', name='Type 1 utility')               
layout2 = go.Layout(
    title=('Utility'),
    width=800,
    height=600,
    xaxis=dict(
        title='round',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Utility',
        nticks=10,
#         range=[],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
data2 = [trace2]
figure2 = go.Figure(data=data2,layout=layout2)
figure2.add_trace(go.Scatter(x=list(range(rounds)), y=list(average_all_runs_type2_average_utility),
                    mode='lines', name='Type 2 utility'))

figure2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
figure2.show()

In [ ]:
### ONLY FOR CHROMOSOME SET UP ###
L = 30                             #chromosome length
type1_chromosome_split = int(L/2)
type1_normalized_parameter = int("1"*type1_chromosome_split,2)/2      #y1 \in [0,2]
type2_normalized_parameter = int("1"*L,2)/2                           #x1 \in [0,2]

def randominitialize(popl_size, chromosome_length):
    pop = []
    for i in range(popl_size):
        string = ''
        for j in range(J):
            string += str(random.randint(0,1))
        pop.append(string)
    return pop

def decoder_type1(pop_chromosomes,chromosome_split):
    decoded_pop = []
    for n in range(len(pop_chromosomes)):
        pair = []
        pair.append(int(pop_chromosomes[n][0:chromosome_split],2))
        pair.append(int(pop_chromosomes[n][chromosome_split:],2))
        decoded_pop.append(pair)
    return decoded_pop

def decoder_type2(pop_chromosomes):
    decoded_pop = []
    for n in range(len(pop_chromosomes)):
        decoded_pop.append(int(pop_chromosomes[n],2))
    return decoded_pop


def normalization_type1(decoded_pop, normalized_parameter):
    normalized_pop = []
    for n in range(len(decoded_pop)):
        pair = []
        pair.append(decoded_pop[n][0]/normalized_parameter)
        if decoded_pop[n][1] % 2 == 0:
            pair.append(0)
        else:
            pair.append(1)
        normalized_pop.append(pair)
    return normalized_pop
    
    
def normalization_type2(decoded_pop, normalized_parameter):
    normalized_pop = []
    for n in range(len(decoded_pop)):
        normalized_pop.append(decoded_pop[n]/normalized_parameter)
    return normalized_pop

### END - ONLY FOR CHROMOSOME SET UP ###
